In [288]:
import pdfminer
import spacy
import pandas as pd
import re
import os
import pdf2txt
from spacy import displacy

# Load LanguageModel from Spacy

In [289]:
nlp = spacy.load("en_core_web_sm")

# Convert_pdf_text

In [290]:
def pdf_to_txt(f_path):
    text_path=os.path.basename(os.path.splitext(f_path)[0])+'.txt'
    output_path=os.path.join('output/txt/',text_path)
    pdf2txt.main(args=[f_path,"--outfile",output_path])
    print("saved sucessfully...")
    return open(output_path).read()

In [291]:
text=pdf_to_txt("resumes/Lauren Safwat.pdf")

saved sucessfully...


In [292]:
x=nlp(text)

In [293]:
displacy.render(x,style='ent',jupyter=True)

In [294]:
names=[]
phone_numbers=[]
skills=[]
mails=[]
no_skills=[]
table={'name':names,'phone_number':phone_numbers,'skills':skills,'no_skills_matched':no_skills,'mail':mails}

In [295]:
def parse_cv(txt,jobSkills):
    jobSkills='|'.join(jobSkills)
   
    skills_pattern=re.compile(jobSkills)
    phone_pattern=re.compile('(\+\d{1,2}\s)?\(?\d{3}\)?[\s.-]\d{3}[\s.-]\d{4}$')
    mail_pattern=re.compile('([\w]*[\w\.]*(?!\.)@gmail.com)')
    name_before_processing=[ent for ent in nlp(txt).ents if ent.label_=='PERSON'][0]
    name=' '.join(str(name_before_processing).split(' ')[0:2])
    
    names.append(name)
    phone_numbers.append([ent for ent in nlp(txt).ents if ent.label_=='CARDINAL'][0]) ##not_yet
    
    mail_found=' '.join(re.findall(mail_pattern,txt.lower()))
    mails.append(mail_found)
    
    skills_matched=re.findall(skills_pattern,txt.lower())
    no_skills.append(len(skills_matched))
    
    skills_matched=' '.join(skills_matched)
    skills.append(skills_matched)
    
    
    
    
    return names,mails,skills,no_skills,phone_numbers

In [296]:
directory_of_cvs='resumes/'
list_of_skills=['pyspark','sklearn','autocad','tensorflow','hadoop','mysql','opencv','html','css']

In [297]:
for file in os.listdir(directory_of_cvs):
    if file.endswith('.pdf'):
        print('Reading.....' + file)
        txt = pdf_to_txt(os.path.join('resumes/',file))
        names,mails,skills,no_skills,phone_numbers=parse_cv(txt,list_of_skills)

Reading.....Lauren Safwat.pdf
saved sucessfully...
Reading.....Mahmoud_Hassan_cv.pdf
saved sucessfully...
Reading.....Omar_Marie_CV.pdf
saved sucessfully...
Reading.....Ali_Said Daebis_Resume.pdf
saved sucessfully...
Reading.....Mohamed_Abdallah_CV (1).pdf
saved sucessfully...


In [298]:
Candidates=pd.DataFrame(table)

In [299]:
Candidates.sort_values(by='no_skills_matched',ascending=False,inplace=True)

In [300]:
Candidates.head()

,name,phone_number,skills,no_skills_matched,mail
3,Ali,(07/2018),mysql sklearn tensorflow opencv pyspark tensor...,11,alid3bs38@gmail.com
0,Lauren Safwat,"(106, 310, 0709)",tensorflow opencv mysql html css,5,lauren.safwat18@gmail.com
1,Mahmoud Hassan\nJunior,(3.42),opencv sklearn mysql pyspark tensorflow,5,mahmoudhassan1644@gmail.com
2,Omar Hamed,"(109, 038)",pyspark sklearn tensorflow html css,5,omarhamed440@gmail.com
4,Mohamed Abdallah,(9),tensorflow pyspark opencv,3,mohamedabdallah9850@gmail.com
